In [1]:
import pandas as pd
import pandana as pdna
import numpy as np
import json

In [2]:
nodes=pd.read_csv('nodes_wgs84.csv')
nodes=nodes.set_index('id')

In [3]:
edges=pd.read_csv('edges_df.csv')

In [4]:
nodes['families']=nodes['type_of_entity']=='FAMILY_OCCUPANCY'
nodes['non_family_hhs']=nodes['type_of_entity'].isin(
    ['QUAD_OCCUPANCY', 'DUAL__OCCUPANCY', 'TRIPLE_OCCUPANCY', 'SINGLE_OCCUPANCY'])
nodes['parks']=nodes['type_of_entity']=='PARKS'
nodes['groceries']=nodes['type_of_entity'].isin(
    ['GROCERY_OR_SUPERMARKET', 'CONVENIENCE_STORE'])
nodes['services']=nodes['type_of_entity'].isin(
    ['BANKS', 'LAUNDRY', 'HEALTH_CARE', 'LIBRARY'])
nodes['eating']=nodes['type_of_entity'].isin(
    ['CAFE', 'RESTAURANT'])
nodes['bars']=nodes['type_of_entity'].isin(
    ['BAR'])

In [5]:
max_dist=400
targets={'parks':{'from':'families' }, 
         'groceries':{'from': 'families'}, 
         'services':{'from': 'families'}, 
         'eating':{'from': 'non_family_hhs'}, 
         'bars':{'from': 'non_family_hhs'}}
target_list=list(targets.keys())

In [6]:
net=pdna.Network(node_x=nodes['lon'], 
                 node_y=nodes['lat'],
                 edge_from=edges['from'], 
                 edge_to=edges['to'], 
                 edge_weights=edges[['distance']])

# Method 1
### Get node to node distance matrix

In [7]:
source_nodes=nodes.loc[nodes['type_of_entity'].str.contains('OCCUPANCY')].reset_index()

In [9]:
nodes['is_target']=nodes[target_list].sum(axis=1)>0

In [10]:
target_nodes=nodes.loc[nodes['is_target']]

In [14]:
dist_mat=[]
for ind, row in source_nodes.iterrows():
    dest_nodes=target_nodes.index
    origin_nodes=[row['id']]*len(dest_nodes)
    path_lengths=net.shortest_path_lengths(origin_nodes, dest_nodes)
    dist_mat.append(list(path_lengths))

In [15]:
dist_mat_np=np.array(dist_mat)

In [17]:
(dist_mat_np<max_dist).astype(int)

array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
reachable_mat=(dist_mat_np<max_dist).astype(int)
access = np.dot(reachable_mat, np.array(target_nodes[target_list]).astype(int))
access_df=pd.DataFrame(access, columns=[t+' prox' for t in target_list])
for col in ['lon', 'lat', 'z']:
    access_df[col]=list(source_nodes[col])
# access_df=pd.concat([access_df, source_nodes[['lon', 'lat', 'z']]], axis=1,
#                    ignore_index=True)

In [51]:
access_df.to_csv('prox_results_1.csv')

In [66]:
metrics={}
for t in target_list:
    from_places=targets[t]['from']
    metrics[t]=(access_df[t+' prox']*source_nodes[from_places]).sum()/(
        source_nodes[from_places].sum())

#### Build a reachable matrix but with the ids to be used for the reachable matrix for the voxelized volpe (Generative tool)

In [ ]:
dist_mat=[]
temporal = ['Origins']
temporal.extend(target_nodes.index)
dist_mat.append(temporal)
for ind, row in source_nodes.iterrows():
    dest_nodes=target_nodes.index
    origin_nodes=[row['id']]*len(dest_nodes)
    path_lengths=net.shortest_path_lengths(origin_nodes, dest_nodes)
    temporal = [row['id']]
    temporal.extend(list(path_lengths))
    dist_mat.append(temporal)

In [ ]:
reachable_mat = dist_mat.copy()

In [ ]:
for row in range(1,len(source_nodes)+1):
    for column in range(1,len(target_nodes)+1):
        boolean = reachable_mat[row][column]<max_dist
        if boolean == False:
            reachable_mat[row][column] = 0
        else:
            reachable_mat[row][column] = 1

In [ ]:
np.savetxt("reachable_mat.csv",reachable_mat, fmt='%s', delimiter=",")

In [ ]:
# TO VERIFY THAT IT WAS PROPERLY CREATED
reachable = (np.genfromtxt("reachable_mat.csv", delimiter=",")).astype(int)

### Method 2

In [24]:
for t in targets:
    target_df=nodes.loc[nodes[t]]
    net.set(target_df.index, variable=1, name=t)

In [25]:
prox_results={}
for t in targets:
    prox_results[t]=net.aggregate(400, type="sum", decay="flat", name=t)

In [26]:
for t in prox_results:
    nodes['{} prox'.format(t)]=prox_results[t]

In [27]:
nodes.to_csv('prox_results_2.csv')